# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,3369157,Cape Town,20.0,190.00,ZA,29131.0,288.25,290.93,292.04,1016.0,77.0,NaN,NaN,-51.968991,-3.848592
1,1,6165406,Thompson,20.0,260.00,CA,24615.0,240.15,246.15,246.15,1023.0,75.0,NaN,NaN,59.750385,-97.258161
2,2,3395981,Maceió,20.0,120.00,BR,29915.0,301.51,299.15,299.15,1013.0,74.0,NaN,NaN,NaN,NaN
3,3,3374210,São Filipe,61.0,56.00,CV,29629.0,295.63,296.29,296.29,1014.0,74.0,1014.0,1009.0,NaN,NaN
4,4,3874787,Punta Arenas,40.0,260.00,CL,2794.0,267.36,278.71,280.15,997.0,65.0,NaN,NaN,-79.100128,-100.457137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,572,2015852,Svetlaya,86.0,297.00,RU,25703.0,248.77,257.03,257.03,1009.0,81.0,1009.0,1008.0,39.219776,44.174836
573,573,3105522,Bereda,98.0,5.81,ES,28036.0,276.55,279.82,281.48,1009.0,99.0,NaN,NaN,70.242092,164.509014
574,574,3449053,São Gotardo,54.0,292.00,BR,29304.0,294.92,293.04,293.04,1012.0,84.0,1012.0,898.0,NaN,NaN
575,575,2352250,Afikpo,90.0,205.00,NG,29826.0,301.48,298.26,298.26,1010.0,85.0,1010.0,998.0,-74.114852,24.920317


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)


In [7]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 4)

# fig.add_layer(heat_layer)

# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 1026.28]
ideal_weather = city_df.loc[city_df['Feels Like'] > 1005.76]
ideal_weather = city_df.loc[city_df['Max Temp'] < 296.5]
ideal_weather = city_df.loc[city_df['Min Temp'] > 291.5]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
61,61,2123814,Leningradskiy,79.0,68.0,RU,26503.0,256.33,265.03,265.03,1023.0,92.0,1023.0,1022.0,-82.050472,14.679931
314,314,1489508,Toguchin,100.0,239.0,RU,26301.0,254.43,263.01,263.01,1026.0,95.0,1026.0,1009.0,-11.952041,-124.377881
360,360,1685615,Suba,98.0,56.0,PH,30173.0,301.88,301.73,301.73,1010.0,77.0,1010.0,1009.0,63.638084,-99.547919
129,129,2110227,Butaritari,57.0,97.0,KI,30085.0,301.11,300.85,300.85,1009.0,78.0,1009.0,1008.0,70.415373,91.321292
459,459,3414079,Olafsvik,99.0,47.0,IS,27659.0,264.60,276.59,276.59,995.0,79.0,995.0,991.0,37.950741,-97.126397
311,311,6167125,Tobermory,99.0,18.0,CA,27215.0,259.46,272.15,272.15,1018.0,74.0,1018.0,994.0,6.183858,24.811060
180,180,2231564,Fontem,91.0,75.0,CM,29341.0,295.73,293.41,293.41,1011.0,88.0,1011.0,915.0,48.793980,-53.309402
477,477,2399870,Lastoursville,100.0,247.0,GA,29598.0,299.75,295.98,295.98,1011.0,90.0,1011.0,984.0,-48.518532,-57.065786
254,254,2015306,Tiksi,67.0,245.0,RU,24169.0,236.08,241.69,241.69,1026.0,82.0,1026.0,1021.0,-3.306316,-7.590204
441,441,1504442,Karatuzskoye,100.0,175.0,RU,26445.0,258.79,264.45,264.45,1030.0,96.0,1030.0,989.0,-43.612932,148.499870


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City", "Latitude":"Lat", "Longitude":"Lng"})
hotel_df

,index,City,Country,Lat,Lng,Hotel Name
0,61,Leningradskiy,RU,-82.050472,14.679931,
1,314,Toguchin,RU,-11.952041,-124.377881,
2,360,Suba,PH,63.638084,-99.547919,
3,129,Butaritari,KI,70.415373,91.321292,
4,459,Olafsvik,IS,37.950741,-97.126397,
5,311,Tobermory,CA,6.183858,24.811060,
6,180,Fontem,CM,48.793980,-53.309402,
7,477,Lastoursville,GA,-48.518532,-57.065786,
8,254,Tiksi,RU,-3.306316,-7.590204,
9,441,Karatuzskoye,RU,-43.612932,148.499870,


In [10]:
# #the latitudes and longitudes I saved from citipy were not getting along with the google api at this point, so I decided
#     #to get the place_id from the api and then get google's preferred coordinates from their own native api in order to 
#     #get to the hotel data.

# #get place_id for city
# placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         placeid_params = {
#         "input": row['City'],
#         "inputtype": "textquery",
#         "key": gkey,
#     }
#         ID = requests.get(placeid_url, params=placeid_params).json()
#         ID = (ID["candidates"][0]["place_id"])
#         print(ID)
    

In [11]:
# #get coordinates from place_id
# coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         coordinate_params = {
#         "place_id": ID,
#         "key": gkey,
#     }
# #     assemble url and make API request
#         coords = requests.get(coordinate_url, params=coordinate_params).json()
#         lat = coords['result']['geometry']['location']['lat']
#         lng = coords['result']['geometry']['location']['lng']
#         coords = f'{lat},{lng}'
#         print(coords)

In [12]:
# find the closest restaurant of each type to coordinates


placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotels = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    print(f'City = {City}')
    

#     use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords_req = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords_req['result']['geometry']['location']['lat']
    lng = coords_req['result']['geometry']['location']['lng']
    try:
        Country = row["Country"]
        print(f'Country = {Country}')
    except:
        print(f'Country = n/a')

    
    coords = f'{lat},{lng}'
    
    hotel_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(hotel_url, params=hotel_params).json()
    # extract results
    try:
        results = response['results'][0]['name']
        hotels.append(results)
        print(f'Hotel = {results}')
    except:
        results = {"No hotel found nearby"}
        hotels.append(results)
        print(results)
    
    
    try:
        print(f"The closest hotel to {City} is {results}.")
    except (KeyError, IndexError):
        print(f"Sorry, there are no hotels within 5000 meters of {City}. Maybe try someplace less remote?")
        
    print("------------")
hotel_df["Hotel Name"] = hotels
hotel_df.rename(columns={"Latitude":"Lat", "Longitude":"Lng"}) #not working???
hotel_df


City = Leningradskiy
Country = RU
Retrieving Results for hotels near Leningradskiy:
Hotel = The Ritz-Carlton, Moscow
The closest hotel to Leningradskiy is The Ritz-Carlton, Moscow.
------------
City = Toguchin
Country = RU
Retrieving Results for hotels near Toguchin:
Hotel = Stolitsa
The closest hotel to Toguchin is Stolitsa.
------------
City = Suba
Country = PH
Retrieving Results for hotels near Suba:
Hotel = Hotel Ignacio
The closest hotel to Suba is Hotel Ignacio.
------------
City = Butaritari
Country = KI
Retrieving Results for hotels near Butaritari:
{'No hotel found nearby'}
The closest hotel to Butaritari is {'No hotel found nearby'}.
------------
City = Olafsvik
Country = IS
Retrieving Results for hotels near Olafsvik:
Hotel = Hotel Olafsvik
The closest hotel to Olafsvik is Hotel Olafsvik.
------------
City = Tobermory
Country = CA
Retrieving Results for hotels near Tobermory:
Hotel = Escarpment Heights Motel
The closest hotel to Tobermory is Escarpment Heights Motel.
-------

,index,City,Country,Lat,Lng,Hotel Name
0,61,Leningradskiy,RU,-82.050472,14.679931,"The Ritz-Carlton, Moscow"
1,314,Toguchin,RU,-11.952041,-124.377881,Stolitsa
2,360,Suba,PH,63.638084,-99.547919,Hotel Ignacio
3,129,Butaritari,KI,70.415373,91.321292,{No hotel found nearby}
4,459,Olafsvik,IS,37.950741,-97.126397,Hotel Olafsvik
5,311,Tobermory,CA,6.183858,24.811060,Escarpment Heights Motel
6,180,Fontem,CM,48.793980,-53.309402,Hyatt Place Charlotte Airport/Tyvola Road
7,477,Lastoursville,GA,-48.518532,-57.065786,Auberge Bon Coin 02
8,254,Tiksi,RU,-3.306316,-7.590204,Umka
9,441,Karatuzskoye,RU,-43.612932,148.499870,{No hotel found nearby}


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map


# Display figure
